In [1]:
from m4_functions import *

song_path = Path(r'C:\Users\ewais\Documents\GitHub\tensor-hero\Experiments\Test_Songs\1. Andy McKee - Ouray\song.ogg')


In [18]:
def centroid(song_path):
    '''input: the song name as string, like: 'song.ogg'
    output: the centroids and onsets


    tone:6 dimentional array: perfect fifth, minor third, and major third each as two-dimensional coordinates. 
    cens:12 dimentional array: major music scale each as 2 dimentional coordinates 

    '''
    y, sr = librosa.load(song_path)
    bandwidth = librosa.feature.spectral_bandwidth(y, sr=sr, hop_length=512)
    
    S, phase = librosa.magphase(librosa.stft(y=y))
    
    
    D = librosa.stft(y)
    len_song = len(y)/sr
    
    #decompose as harmony, percussion, and residue
    H, P = librosa.decompose.hpss(D, margin=3.0)
    #eqation between the harmony,residue and percussion
    R = D - (H+P)
    
    y_harm = librosa.istft(H)
    y_perc = librosa.istft(P)
    #the residue is less focus on the vocal more on the string, however the drums are more in focus comparing to harmony
    y_resi = librosa.istft(R)
    
    #onset_time
    onset_frame_backtrack = librosa.onset.onset_detect(y_harm, sr = sr, hop_length = 512, backtrack=True)
    onset_times = librosa.frames_to_time(onset_frame_backtrack)
    
    #outputting the centroids
    cent =librosa.feature.spectral_centroid(S=S)

    
    # the major scale x and y
    x = cent[0]
    bw = bandwidth[0]
    #y = tonnetz[5]
    
    #the number of tonnetz happens per sec
    elip = len_song /len(x) 
    time = [elip*i for i in range(len(x))]
    
    #the threshold for the onset time
    threshold = 0.005
    # index of the tonnetz time
    #inx = []
    # final value output
    centroids = []
    bandw = []

    for i in range(len(onset_times)):
        target = onset_times[i]

        for j in range (len(time)):
            diff = (target - time[j])
            if  diff < threshold:
                #inx.append(j)
                centroids.append(x[j])
                bandw.append(bw[j])
                break  

    return centroids, onset_times, bandw

centroids, onset_times, bandwidth = centroid(str(song_path))



In [19]:
print(len(centroids))
print(onset_times.shape)
print(len(bandwidth))

653
(653,)
653


In [ ]:
print(min(bandwidth))
print(max(bandwidth))
print(bandwidth)

In [28]:

def generate_centroid_notes(onset, centroids, bandwidth, interval_length=100):
    '''[summary]

    Args:
        onset ([type]): [description]
        centroid ([type]): [description]
        interval_length (int, optional): [description]. Defaults to 100.

    Returns:
        [type]: [description]
    '''
    # Notes: 1-5 are single notes
    # 6 - 31 are chords
    # 32 is open note
    # conditions:
    # check onset and onset + 1
    # if < 2 then it should be single note
    notes = [5]
    repeat_threshold = 200
    
    # centroid format is 10ms bins, each index corresponds to 10ms of time
    for i in range(1, len(centroids)):
        diff = centroids[i] - centroids[i-1]
        if abs(diff) < abs(repeat_threshold):
            notes.append(notes[i-1])
        else:
            if diff > 0:
                notes.append(notes[i-1] + 1)
            else:
                notes.append(notes[i-1] -1)
    
    notes = [(x%5)+1 for x in notes]
    
    for i in range(len(bandwidth)):
        if (bandwidth[i] > 2600) & (bandwidth[i-1] > 2600) & (i > 0):
            notes[i] = notes[i-1]
        elif (bandwidth[i] > 2600):
            notes[i] = random.randint(6,31)
   
    return notes


def generate_song(song_path, 
                  note_generation_function = generate_centroid_notes, 
                  onset_computation_function = centroid,
                  generation_function_uses_song_as_input = False, 
                  source_separated_path = None, 
                  outfile_song_name = 'Model 4 - centroids', 
                  artist = 'Forrest',
                  outfolder = None,
                  original_song_path = None):
    '''
    Takes the song present at song_path, uses onset_computation_function to compute onsets, uses note_generation_function
    to generate notes, then writes the song to an outfolder at ~/Model_3/Generated Songs/<outfile_song_name>

    ~~~~ ARGUMENTS ~~~~
    - song_path : Path or str
        - path to the original song, i.e. song.ogg
        - this will be used to create the folder ingested by Clone Hero
    - note_generation_function : function
        - takes onset indices as input, formatted as an array of onset times in 10ms time bins
        - can optionally take song data as input, should set generation_function_uses_song_as_input = True in this case
        - outputs a sequence of notes that has the same length as the onsets array
        - uses source separated audio by default
    - onset_computation_function : function
        - given song_path (or source_separated_path if it is not None), generates onsets
        - onsets should be formatted as an array with entries being onsets in seconds, these will later be converted to
          the 10ms time bines required by note_generation_function
    - generation_function_uses_song_as_input : bool
        - if True, note_generation_function will be called as note_generation_function(onset_indices, song_path)
        - if False, note_generation_function will be called as note_generation_function(onset_indices)
    - source_separated_path : Path or str
        - Path to source separated file if it exists
    - outfile_song_name : str
        - this will determine the name of the folder the song will be saved under
        - it also determines the song name that will appear in Clone Hero once the folder is transferred
    - artist : str
        - determines the artist written to the .chart file
    - outfolder : Path
        - Determines the folder where the generated song will be held
    - original_song_path : Path
        - if not none, will be used to save the original song rather than source separated song
          to outfolder
    '''
    if source_separated_path is not None:
        path = source_separated_path
    else:
        path = song_path

    print('Computing onsets...')
    centroids, onset_times, bandwidth = onset_computation_function(str(path))
    onset_indices = onset_time_bins(onset_times)

    print('Generating Notes...\n')
    if generation_function_uses_song_as_input:
        dense_notes = note_generation_function(onset_indices[:-1], path)
    else:
        dense_notes = note_generation_function(onset_indices[:-1], centroids, bandwidth)
    
    notes_array = create_notes_array(onset_indices[:len(dense_notes)], dense_notes)

    song_metadata = {'Name' : outfile_song_name,
                    'Artist' : artist,
                    'Charter' : 'tensorhero',
                    'Offset' : 0,
                    'Resolution' : 192,
                    'Genre' : 'electronic',
                    'MediaType' : 'cd',
                    'MusicStream' : 'song.ogg'}

    # outfolder = Path(r'C:\Users\ewais\Documents\GitHub\tensor-hero\Model_3\Generated Songs\\'+ outfile_song_name)
    write_song_from_notes_array(song_metadata, notes_array, outfolder)
    if original_song_path is not None:
        shutil.copyfile(str(original_song_path), str(outfolder / 'song.ogg'))
    else:
        shutil.copyfile(str(path), str(outfolder / 'song.ogg'))
    return notes_array



In [29]:
generate_song(song_path=song_path,
              outfolder=Path(r'C:\Users\ewais\Documents\GitHub\tensor-hero\Model_3\test_song'))

Computing onsets...
Generating Notes...



array([0., 0., 0., ..., 0., 0., 0.])